In [12]:
import pandas as pd
import numpy as np
from data import result_df
import plotly.express as px
import warnings
warnings.filterwarnings('ignore', category=UserWarning)


In [13]:
# Assign the DataFrame from the imported module to df
df = result_df

In [14]:
# Crear una tabla de contingencia
contingency_table = pd.crosstab(df['HTR'], df['FTR'])

# Resetear los índices para plotly express
contingency_table.reset_index(inplace=True)

# Plot
fig = px.bar(contingency_table, x='HTR', y=list(contingency_table.columns[1:]),
             title='Distribución de Resultados al Medio Tiempo y al Final del Partido',
             labels={'halftime_result': 'Resultado al Medio Tiempo', 'value': 'Frecuencia', 'variable': 'Resultado al Final del Partido'},
             barmode='stack')

# fig = px.imshow(contingency_table.values,
#                 labels=dict(x="FTR", y="HTR", color="Frecuencia"),
#                 x=contingency_table.columns, y=contingency_table.index,
#                 title="Distribución de Resultados al Medio Tiempo y al Final del Partido",
#                 color_continuous_scale="viridis")

fig.show()

In [15]:

# Get unique teams
teams = df['HomeTeam'].unique()

# Create a dictionary with teams and initial values
team_stats = {team: {'Points': 0, 'GoalsFor': 0, 'GoalsAgainst': 0, 'GoalDifference': 0,
                     'Wins': 0, 'Draws': 0, 'Losses': 0} for team in teams}

# Iterate over each row of the original DataFrame
for index, row in df.iterrows():
    # Check if the team was HomeTeam or AwayTeam and update statistics
    home_team = row['HomeTeam']
    away_team = row['AwayTeam']
    home_goals = row['FTHG']
    away_goals = row['FTAG']

    # Update statistics
    team_stats[home_team]['GoalsFor'] += home_goals
    team_stats[home_team]['GoalsAgainst'] += away_goals
    team_stats[away_team]['GoalsFor'] += away_goals
    team_stats[away_team]['GoalsAgainst'] += home_goals

    # Assign points based on FTR
    if row['FTR'] == 'H':
        team_stats[home_team]['Points'] += 3
        team_stats[home_team]['Wins'] += 1
        team_stats[away_team]['Losses'] += 1
    elif row['FTR'] == 'A':
        team_stats[away_team]['Points'] += 3
        team_stats[home_team]['Losses'] += 1
        team_stats[away_team]['Wins'] += 1
    elif row['FTR'] == 'D':
        team_stats[home_team]['Points'] += 1
        team_stats[away_team]['Points'] += 1
        team_stats[home_team]['Draws'] += 1
        team_stats[away_team]['Draws'] += 1


In [16]:
# Calculate Goal Difference for each team
for team, stats in team_stats.items():
    stats['GoalDifference'] = stats['GoalsFor'] - stats['GoalsAgainst']

# Sort the dictionary by points from highest to lowest
sorted_team_stats = dict(sorted(team_stats.items(), key=lambda item: item[1]['Points'], reverse=True))

# Convert the sorted_team_stats to a DataFrame for better table formatting
df_stats = pd.DataFrame.from_dict(sorted_team_stats, orient='index')

# Display the sorted team statistics as a table
# print(tabulate(df_stats, headers='keys', tablefmt='pretty'))


In [17]:
df_stats['Avg_Goals_Scored'] = df_stats['GoalsFor'] / 38
df_stats['Avg_Goals_Conceded'] = df_stats['GoalsAgainst'] / 38
df_stats

,Points,GoalsFor,GoalsAgainst,GoalDifference,Wins,Draws,Losses,Avg_Goals_Scored,Avg_Goals_Conceded
Man City,93,99,26,73,29,6,3,2.605263,0.684211
Liverpool,92,94,26,68,28,8,2,2.473684,0.684211
Chelsea,74,76,33,43,21,11,6,2.000000,0.868421
Tottenham,71,69,40,29,22,5,11,1.815789,1.052632
Arsenal,69,61,48,13,22,3,13,1.605263,1.263158
Man United,58,57,57,0,16,10,12,1.500000,1.500000
West Ham,56,60,51,9,16,8,14,1.578947,1.342105
Leicester,52,62,59,3,14,10,14,1.631579,1.552632
Brighton,51,42,44,-2,12,15,11,1.105263,1.157895
Wolves,51,38,43,-5,15,6,17,1.000000,1.131579


In [21]:
# Crear el gráfico de burbujas

fig = px.scatter(df_stats, x="Avg_Goals_Scored", y="Avg_Goals_Conceded", text="Team", size_max=60)

# Personalizar el gráfico (opcional)
fig.update_traces(marker=dict(line=dict(width=2, color='DarkSlateGrey')), selector=dict(mode='markers+text'))

# Mostrar el gráfico
fig.show()

ValueError: Value of 'text' is not the name of a column in 'data_frame'. Expected one of ['Points', 'GoalsFor', 'GoalsAgainst', 'GoalDifference', 'Wins', 'Draws', 'Losses', 'Avg_Goals_Scored', 'Avg_Goals_Conceded', 'Bubble_Size'] but received: Team

In [22]:
# Calcular el tamaño de las burbujas según el ratio
df_stats["Bubble_Size"] = df_stats["Avg_Goals_Scored"] / df_stats["Avg_Goals_Conceded"]

# Crear el gráfico de burbujas
fig = px.scatter(df_stats, x="Avg_Goals_Scored", y="Avg_Goals_Conceded", text="Team", size="Bubble_Size", size_max=60,
                 color=df_stats["Team"].apply(lambda x: "Top 5" if x in ["Man City", "Liverpool", "Chelsea", "Tottenham", "Arsenal"] else ("Bottom 5" if x in ["Norwich", "Watford", "Burnley", "Leeds", "Everton"] else "Others")))

# Personalizar el gráfico (opcional)
fig.update_traces(marker=dict(line=dict(width=2, color='DarkSlateGrey')), selector=dict(mode='markers+text'))

# Mostrar el gráfico
fig.show()

KeyError: 'Team'

In [23]:

# Reset index
df_stats.reset_index(inplace=True)  # Resetear el índice
df_stats = df_stats.rename(columns={'index': 'Team'})
Total_goals = df_stats['GoalsFor'].sum()
Avg_goals_per_game = Total_goals / (df.count()[0])

df_Total_goals_by_match = df['FTHG'] + df['FTAG']

df_consolidated = df_Total_goals_by_match.to_frame(name='TotalGoalsByMatch')
df_consolidated['RedCards'] = df['HR'] + df['AR']
df_consolidated['YellowCards'] = df['HY'] + df['AY']
df_consolidated['Corners'] = df['HC'] + df['AC']

df_consolidated.head(5)
type(df_consolidated)


df_wins_rate_home = df['FTR'].value_counts().get('H',0) / (df.count()[0])

# Referee stats
referee_win_rate = df.groupby("Referee").apply(lambda x: (x["FTR"] == "H").mean() * 100).sort_values(ascending=False)
nombres_arbitros = referee_win_rate.index
porcentajes = referee_win_rate.values
nombres_arbitros[0]
porcentajes[0]

# original data
original = df

In [24]:
df_consolidated

,TotalGoalsByMatch,RedCards,YellowCards,Corners
0,2,0,0,7
1,6,0,3,9
2,3,0,3,13
3,3,0,0,7
4,4,0,2,14
...,...,...,...,...
375,1,0,6,9
376,5,0,1,6
377,4,0,1,8
378,5,0,1,14


In [28]:
fig = px.box(df_consolidated, labels={'variable': 'Variables', 'value': 'Valores'},orientation='h', points='all')

fig.show()


In [ ]:
team_selected = 'Leeds'
perf_by_team_selected = df.query("HomeTeam == @team_selected | AwayTeam == @team_selected")

In [ ]:
def calculate_points(row):
    if row['FTR'] == 'H':
        return 3 if row['HomeTeam'] == team_selected else 0
    elif row['FTR'] == 'A':
        return 3 if row['AwayTeam'] == team_selected else 0
    elif row['FTR'] == 'D':
        return 1 if row['HomeTeam'] == team_selected or row['AwayTeam'] == team_selected else 0
    else:
        return 0

perf_by_team_selected['Points'] = perf_by_team_selected.apply(calculate_points, axis=1)
perf_by_team_selected['Cumul_Points'] = perf_by_team_selected['Points'].cumsum()




/tmp/ipykernel_8423/3213345856.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  perf_by_team_selected['Points'] = perf_by_team_selected.apply(calculate_points, axis=1)
/tmp/ipykernel_8423/3213345856.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  perf_by_team_selected['Cumul_Points'] = perf_by_team_selected['Points'].cumsum()


In [ ]:
conditions = [
    (perf_by_team_selected['Points'] == 3),
    (perf_by_team_selected['Points'] == 0),
    (perf_by_team_selected['Points'] == 1)
]

values = ['Won', 'Lose', 'Draw']

perf_by_team_selected['Result'] = np.select(conditions, values)

perf_by_team_selected

/tmp/ipykernel_8423/678775143.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  perf_by_team_selected['Result'] = np.select(conditions, values)


,Date,HomeTeam,AwayTeam,FTHG,FTAG,FTR,HTHG,HTAG,HTR,Referee,...,AF,HC,AC,HY,AY,HR,AR,Points,Cumul_Points,Result
1,14/08/2021,Man United,Leeds,5,1,H,1,0,H,P Tierney,...,9,5,4,1,2,0,0,0,0,Lose
13,21/08/2021,Leeds,Everton,2,2,D,1,1,D,D England,...,13,8,5,2,4,0,0,1,1,Draw
27,29/08/2021,Burnley,Leeds,1,1,D,0,0,D,M Oliver,...,9,5,5,4,3,0,0,1,2,Draw
38,12/09/2021,Leeds,Liverpool,0,3,A,0,1,A,C Pawson,...,8,2,11,2,1,1,0,0,2,Lose
40,17/09/2021,Newcastle,Leeds,1,1,D,1,1,D,M Dean,...,11,4,5,2,2,0,0,1,3,Draw
53,25/09/2021,Leeds,West Ham,1,2,A,1,0,H,K Friend,...,6,5,7,3,2,0,0,0,3,Lose
63,02/10/2021,Leeds,Watford,1,0,H,1,0,H,S Hooper,...,16,6,3,0,3,0,0,3,6,Won
75,16/10/2021,Southampton,Leeds,1,0,H,0,0,D,D Coote,...,14,8,1,2,3,0,0,0,6,Lose
84,23/10/2021,Leeds,Wolves,1,1,D,0,1,A,R Jones,...,14,6,3,2,3,0,0,1,7,Draw
97,31/10/2021,Norwich,Leeds,1,2,A,0,0,D,A Taylor,...,23,5,0,1,4,0,0,3,10,Won


In [ ]:
#%pip install plotly
#%pip install nbformat>=4.2.0
#import plotly.express as px

# # Create the scatter plot using the filtered data
# scatter_fig = px.scatter(perf_by_team_selected, x="Date", y="Cumul_Points")

# # Show the scatter plot
# scatter_fig.show()

# # Create the bar plot for the Result column
# bar_fig = px.bar(perf_by_team_selected, x="Result", y="Result", color="Result", title="Match Results")

# # Show the bar plot
# bar_fig.show()


In [ ]:
# perf_by_team_selected['Date'] = pd.to_datetime(perf_by_team_selected['Date'], dayfirst=True)

# # Filtrar los datos para obtener solo las victorias, derrotas y empates
# wins = perf_by_team_selected[perf_by_team_selected["Result"] == "Won"].groupby("Date").size().cumsum()
# losses = perf_by_team_selected[perf_by_team_selected["Result"] == "Lose"].groupby("Date").size().cumsum()
# draws = perf_by_team_selected[perf_by_team_selected["Result"] == "Draw"].groupby("Date").size().cumsum()

# # Crear el gráfico de líneas
# fig = go.Figure()

# # Añadir la línea para las victorias
# fig.add_trace(go.Scatter(x=wins.index, y=wins.values, mode='lines', name='Wins'))

# # Añadir la línea para las derrotas
# fig.add_trace(go.Scatter(x=losses.index, y=losses.values, mode='lines', name='Losses'))

# # Añadir la línea para los empates
# fig.add_trace(go.Scatter(x=draws.index, y=draws.values, mode='lines', name='Draws'))

# # Establecer el título y etiquetas de los ejes
# fig.update_layout(title='Cumulative Progression of Wins, Losses, and Draws Over Time',
#                    xaxis_title='Date',
#                    yaxis_title='Cumulative Count')

# # Mostrar el gráfico
# fig.show()

In [ ]:
# Calcular porcentajes acumulados
perf_by_team_selected["Cumul_Wins"] = perf_by_team_selected["Result"].apply(lambda x: 1 if x == "Won" else 0).cumsum()
perf_by_team_selected["Cumul_Losses"] = perf_by_team_selected["Result"].apply(lambda x: 1 if x == "Lose" else 0).cumsum()  # Corregido aquí
perf_by_team_selected["Cumul_Draws"] = perf_by_team_selected["Result"].apply(lambda x: 1 if x == "Draw" else 0).cumsum()

perf_by_team_selected['games'] = 1
perf_by_team_selected['games2'] = perf_by_team_selected['games'].cumsum()



/tmp/ipykernel_8423/2967487282.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  perf_by_team_selected["Cumul_Wins"] = perf_by_team_selected["Result"].apply(lambda x: 1 if x == "Won" else 0).cumsum()
/tmp/ipykernel_8423/2967487282.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  perf_by_team_selected["Cumul_Losses"] = perf_by_team_selected["Result"].apply(lambda x: 1 if x == "Lose" else 0).cumsum()  # Corregido aquí
/tmp/ipykernel_8423/2967487282.py:4: SettingWithCopyWarning: 
A value is trying to be s

In [ ]:
perf_by_team_selected

,Date,HomeTeam,AwayTeam,FTHG,FTAG,FTR,HTHG,HTAG,HTR,Referee,...,HR,AR,Points,Cumul_Points,Result,Cumul_Wins,Cumul_Losses,Cumul_Draws,games,games2
1,14/08/2021,Man United,Leeds,5,1,H,1,0,H,P Tierney,...,0,0,0,0,Lose,0,1,0,1,1
13,21/08/2021,Leeds,Everton,2,2,D,1,1,D,D England,...,0,0,1,1,Draw,0,1,1,1,2
27,29/08/2021,Burnley,Leeds,1,1,D,0,0,D,M Oliver,...,0,0,1,2,Draw,0,1,2,1,3
38,12/09/2021,Leeds,Liverpool,0,3,A,0,1,A,C Pawson,...,1,0,0,2,Lose,0,2,2,1,4
40,17/09/2021,Newcastle,Leeds,1,1,D,1,1,D,M Dean,...,0,0,1,3,Draw,0,2,3,1,5
53,25/09/2021,Leeds,West Ham,1,2,A,1,0,H,K Friend,...,0,0,0,3,Lose,0,3,3,1,6
63,02/10/2021,Leeds,Watford,1,0,H,1,0,H,S Hooper,...,0,0,3,6,Won,1,3,3,1,7
75,16/10/2021,Southampton,Leeds,1,0,H,0,0,D,D Coote,...,0,0,0,6,Lose,1,4,3,1,8
84,23/10/2021,Leeds,Wolves,1,1,D,0,1,A,R Jones,...,0,0,1,7,Draw,1,4,4,1,9
97,31/10/2021,Norwich,Leeds,1,2,A,0,0,D,A Taylor,...,0,0,3,10,Won,2,4,4,1,10


In [ ]:
# Calcular porcentajes acumulados como fracción del total
perf_by_team_selected["Cumul_Wins_Percentage"] = (perf_by_team_selected["Cumul_Wins"] / perf_by_team_selected['games2']) * 100
perf_by_team_selected["Cumul_Losses_Percentage"] = (perf_by_team_selected["Cumul_Losses"] / perf_by_team_selected['games2']) * 100
perf_by_team_selected["Cumul_Draws_Percentage"] = (perf_by_team_selected["Cumul_Draws"] / perf_by_team_selected['games2']) * 100

/tmp/ipykernel_8423/932288519.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  perf_by_team_selected["Cumul_Wins_Percentage"] = (perf_by_team_selected["Cumul_Wins"] / perf_by_team_selected['games2']) * 100
/tmp/ipykernel_8423/932288519.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  perf_by_team_selected["Cumul_Losses_Percentage"] = (perf_by_team_selected["Cumul_Losses"] / perf_by_team_selected['games2']) * 100
/tmp/ipykernel_8423/932288519.py:4: SettingWithCopyWarning: 
A value is trying to be set o

In [ ]:
perf_by_team_selected

,Date,HomeTeam,AwayTeam,FTHG,FTAG,FTR,HTHG,HTAG,HTR,Referee,...,Cumul_Points,Result,Cumul_Wins,Cumul_Losses,Cumul_Draws,games,games2,Cumul_Wins_Percentage,Cumul_Losses_Percentage,Cumul_Draws_Percentage
1,14/08/2021,Man United,Leeds,5,1,H,1,0,H,P Tierney,...,0,Lose,0,1,0,1,1,0.000000,100.000000,0.000000
13,21/08/2021,Leeds,Everton,2,2,D,1,1,D,D England,...,1,Draw,0,1,1,1,2,0.000000,50.000000,50.000000
27,29/08/2021,Burnley,Leeds,1,1,D,0,0,D,M Oliver,...,2,Draw,0,1,2,1,3,0.000000,33.333333,66.666667
38,12/09/2021,Leeds,Liverpool,0,3,A,0,1,A,C Pawson,...,2,Lose,0,2,2,1,4,0.000000,50.000000,50.000000
40,17/09/2021,Newcastle,Leeds,1,1,D,1,1,D,M Dean,...,3,Draw,0,2,3,1,5,0.000000,40.000000,60.000000
53,25/09/2021,Leeds,West Ham,1,2,A,1,0,H,K Friend,...,3,Lose,0,3,3,1,6,0.000000,50.000000,50.000000
63,02/10/2021,Leeds,Watford,1,0,H,1,0,H,S Hooper,...,6,Won,1,3,3,1,7,14.285714,42.857143,42.857143
75,16/10/2021,Southampton,Leeds,1,0,H,0,0,D,D Coote,...,6,Lose,1,4,3,1,8,12.500000,50.000000,37.500000
84,23/10/2021,Leeds,Wolves,1,1,D,0,1,A,R Jones,...,7,Draw,1,4,4,1,9,11.111111,44.444444,44.444444
97,31/10/2021,Norwich,Leeds,1,2,A,0,0,D,A Taylor,...,10,Won,2,4,4,1,10,20.000000,40.000000,40.000000


In [ ]:
# Crear el gráfico de barras apiladas
fig = px.bar(perf_by_team_selected, x="Date", y=["Cumul_Wins_Percentage", "Cumul_Losses_Percentage", "Cumul_Draws_Percentage"],
             title="Porcentaje acumulado de Victorias, Derrotas y Empates",
             labels={"value": "Porcentaje acumulado", "variable": "Resultado"},
             color_discrete_map={"Cumul_Wins_Percentage": "green", "Cumul_Losses_Percentage": "red", "Cumul_Draws_Percentage": "yellow"},
             barmode="stack",
             # text=perf_by_team_selected['Cumul_Wins_Percentage', "Cumul_Losses_Percentage", "Cumul_Draws_Percentage"].apply(lambda x: '{0:1.2f}%'.format(x))
             )

fig.update_xaxes(tickangle=45)
fig.show()

NameError: name 'px' is not defined

In [ ]:
from plotly.subplots import make_subplots

fig = make_subplots(specs=[[{"secondary_y": True}]])